# Analisi e Implementazione di Metodi di Ottimizzazione

Questo notebook implementa e confronta tre metodi di ottimizzazione non vincolata per trovare i minimi di diverse funzioni test:
1.  **Metodo del Gradiente (Gradient Descent - GD)**
2.  **Metodo di Newton**
3.  **Metodo del Gradiente Stocastico (SGD)** - *Implementazione semplificata*

Viene utilizzata una strategia di **Backtracking Line Search** per la determinazione del passo di apprendimento (alpha) in tutti i metodi.

## 1. Import delle librerie

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import numpy.linalg as LA # Per algebra lineare (es. solve)

# matplotlib.use("TkAgg") # Decommentare se si usa un ambiente script, solitamente non necessario in notebook
from mpl_toolkits.mplot3d import Axes3D # Per grafici 3D

## 2. Implementazione degli Algoritmi

### 2.1 Funzione Backtracking (Line Search)

Questa funzione implementa la ricerca del passo $\alpha$ tramite backtracking, basandosi sulla condizione di Armijo per garantire una discesa sufficiente.

In [ ]:
# Backtracking per alpha (Condizione di Armijo)
def backtracking(f, df, X_k, p_k):
    """
    Implementa il backtracking line search per trovare un passo alpha idoneo.
    
    Parametri:
    f: La funzione obiettivo
    df: Il gradiente della funzione obiettivo
    X_k: Il punto corrente (vettore)
    p_k: La direzione di discesa (vettore)
    """
    alpha = 1.0
    rho = 0.5  # Fattore di riduzione per alpha
    c = 0.25   # Costante per la condizione di Armijo

    # Calcola il gradiente nel punto corrente
    grad_k = df(X_k)

    # Calcola il prodotto scalare (grad_k^T * p_k)
    grad_dot_p = np.dot(grad_k, p_k)
    
    # Condizione Di Armijo: Il loop "while" continua FINCHÉ la condizione è VIOLATA
    while f(X_k + alpha * p_k) > f(X_k) + c * alpha * grad_dot_p:
        alpha = rho * alpha

    return alpha

### 2.2 Metodo del Gradiente (GD)

In [ ]:
# Metodo del gradiente
def GD (f, df, X_old, maxit, tol_f, tol_x):
    
    count = 0
    dim = len(X_old)

    # Array per memorizzare la cronologia dei punti visitati
    fun_history = np.zeros((maxit + 1, dim))
    fun_history[0, :] = X_old

    exit_flag = 'maxit' # Flag di uscita di default

    current_grad_norm = np.linalg.norm(df(X_old))

    # Ciclo principale: itera fino a maxit o fino a convergenza
    while count < maxit and current_grad_norm > tol_f:
        # Direzione di discesa = anti-gradiente
        p_k = -df(X_old)

        # Calcolo del passo alpha con backtracking
        alpha = backtracking(f, df, X_old, p_k)

        # Calcolo del nuovo punto X_k+1
        X_new = X_old + alpha * p_k

        # Controllo tolleranza sulla distanza tra iterati (tol_x)
        if np.linalg.norm(X_new - X_old) < tol_x:
            exit_flag = 'tol_x'
            break

        # Ricalcolo la norma del gradiente per il prossimo ciclo
        current_grad_norm = np.linalg.norm(df(X_new))

        # Aggiorno per l'iterazione successiva
        X_old = X_new
        count += 1

        # Salvo il punto corrente nella cronologia
        fun_history[count, :] = X_new

    # Controllo se l'uscita è dovuta a tol_f (norma del gradiente)
    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    # Rimuovo le righe non utilizzate dalla cronologia
    fun_history = fun_history[:count+1]

    return X_old, count, fun_history, exit_flag

### 2.3 Metodo di Newton

Questo metodo utilizza l'informazione del secondo ordine (matrice Hessiana) per calcolare la direzione di discesa, risolvendo il sistema lineare $H_k p_k = -\nabla f_k$.

In [ ]:
# Metodo di Newton Puro
def Newton (f, df, hess_f, X_old, maxit, tol_f, tol_x):
    
    count = 0
    dim = len(X_old)

    # Array per la cronologia
    fun_history = np.zeros((maxit + 1, dim))
    fun_history[0, :] = X_old

    exit_flag = 'maxit'

    current_grad_norm = np.linalg.norm(df(X_old))

    while count < maxit and current_grad_norm > tol_f:

        # Calcolo Gradiente e Hessiana
        grad_k = df(X_old)
        H_k = hess_f(X_old)

        # Direzione di discesa: risolvo il sistema H_k * p_k = -grad_k
        try:
            p_k = LA.solve(H_k, -grad_k)
        except LA.LinAlgError: 
            # Se l'Hessiana è singolare, il metodo fallisce
            print("Errore: Matrice Hessiana Singolare.")
            exit_flag = 'Hessian Error'
            break

        # Backtracking di alpha
        alpha = backtracking(f, df, X_old, p_k)

        # Calcolo di X_k+1
        X_new = X_old + alpha * p_k

        # Controllo tolleranza x
        if np.linalg.norm(X_new - X_old) < tol_x:
            exit_flag = 'tol_x'
            break

        # Ricalcolo la norma del gradiente
        current_grad_norm = np.linalg.norm(df(X_new))

        # Aggiorno per l'iterazione successiva
        X_old = X_new
        count += 1
        fun_history[count, :] = X_new

    
    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    # Rimuovo le righe non utilizzate
    fun_history = fun_history[:count+1]

    return X_old, count, fun_history, exit_flag

### 2.4 Metodo del Gradiente Stocastico (SGD)

**Nota:** L'implementazione seguente è solo una *struttura* per SGD. Per come sono definite le funzioni obiettivo (es. Rosenbrock), non c'è una dipendenza da un dataset $S_k$. Per questo motivo, il calcolo del gradiente `df(X_old)` è in realtà un gradiente *completo* (Full Batch). L'algoritmo si comporta quindi in modo identico al Metodo del Gradiente standard.

In [ ]:
# Metodo del gradiente stocastico (Simulato)
def SGD (f, df, X_old, max_steps, tol_f, tol_x):
    
    # --- Inizializzazione Dati (per un VERO SGD) ---
    # Dimensione del mini-batch (fittizia in questo caso)
    k = 3 
    # Inizializziamo un dataset fittizio di indici
    S_k = np.arange(0, 20, 1)
    # ------------------------------------------------

    # Calcolo la dimensione di X_old
    dim = len(X_old)

    step_count = 0

    # Dichiarare una matrice per la cronologia
    fun_history = np.zeros((max_steps + 1, dim))
    fun_history[0, :] = X_old

    exit_flag = 'maxit'

    current_grad_norm = np.linalg.norm(df(X_old))

    # Il 'while' loop esterno rappresenta le EPOCHE
    while step_count < max_steps and current_grad_norm > tol_f:

        # Randomizziamo gli indici del dataset (shuffle)
        np.random.shuffle(S_k)

        # Il 'for' loop interno itera sui MINI-BATCH
        for i in range (0, len(S_k), k):

            # Estrae gli indici per il mini-batch corrente
            indices = S_k[i:i+k]
            
            # --- CALCOLO GRADIENTE ---
            # NOTA BENE: Questa implementazione NON È STOCASTICA.
            # La variabile 'indices' non viene usata.
            # 'df(X_old)' calcola il gradiente COMPLETO (Full Batch).
            # Per un VERO SGD, si dovrebbe calcolare df(X_old, indices)
            grad_stoc = df(X_old) 
            p_k = -grad_stoc
            # -------------------------

            # Backtracking per alpha
            alpha = backtracking(f, df, X_old, p_k)

            # Calcolo di X_k+1
            X_new = X_old + alpha * p_k

            # Controllo tolleranza x (progresso minimo)
            if np.linalg.norm(X_new - X_old) < tol_x:
                exit_flag = 'tol_x'
                break

            # Aggiorno per l'iterazione successiva
            X_old = X_new
            step_count += 1
            
            # Aggiungo alla cronologia
            if step_count <= max_steps:
                fun_history[step_count, :] = X_new

            # Controllo se ho superato il numero di passi totali (non epoche)
            if step_count >= max_steps:
                break # Interrompe il 'for' loop (mini-batch)

        # Ricalcolo la norma del gradiente completo (per condizione d'arresto)
        current_grad_norm = np.linalg.norm(df(X_new))

        # Se uno dei break interni è stato attivato, esci anche dal 'while'
        if exit_flag == 'tol_x' or step_count >= max_steps:
            break

    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    # Tronchiamo l'array della cronologia
    fun_history = fun_history[:step_count+1]

    return X_old, step_count, fun_history, exit_flag

## 3. Funzione per la Generazione dei Grafici

In [ ]:
def graph_generator (x_range, y_range, f, path_history, tmin, title_2d="Percorso Ottimizzazione", title_3d="Superficie 3D"):
    """
    Genera due grafici affiancati: un contour plot 2D e un surface plot 3D.
    
    Parametri:
    x_range: Range di valori per l'asse x (linspace)
    y_range: Range di valori per l'asse y (linspace)
    f: La funzione obiettivo
    path_history: Matrice (N_iter, 2) con la cronologia dei punti X_k
    tmin: Coordinate (x, y) del minimo teorico
    title_2d: Titolo per il grafico 2D
    title_3d: Titolo per il grafico 3D
    """
    
    X_mesh, Y_mesh = np.meshgrid(x_range, y_range)
    Z = f([X_mesh, Y_mesh])

    # Estrae il percorso delle coordinate x e y
    path_x = path_history[:, 0]
    path_y = path_history[:, 1]

    # Definisco la figura
    fig = plt.figure(figsize=(16, 8))

    # --- GENERAZIONE GRAFICO 3D (a sinistra) ---
    axs1 = fig.add_subplot(1, 2, 1, projection='3d')
    
    # Disegno la superficie
    axs1.plot_surface(X_mesh, Y_mesh, Z, cmap='viridis', rstride=1, cstride=1, linewidth=0, antialiased=True, alpha=0.7)

    # Aggiunge il percorso anche nel grafico 3D
    z_path = f([path_x, path_y])
    axs1.plot(path_x, path_y, z_path, 'r-o', markersize=3, label='Percorso Ottimizzazione')

    # Evidenzia il punto iniziale e finale
    axs1.plot([path_x[0]], [path_y[0]], [z_path[0]], 'go', markersize=10, label=f'Start: ({path_x[0]:.1f}, {path_y[0]:.1f})')
    axs1.plot([path_x[-1]], [path_y[-1]], [z_path[-1]], 'rx', markersize=12, label=f'End: ({path_x[-1]:.2f}, {path_y[-1]:.2f})')
    axs1.plot([tmin[0]], [tmin[1]], [f(tmin)], 'b*', markersize=15, label=f'Minimo Teorico ({tmin[0]}, {tmin[1]})')

    # Aggiungo label e titolo
    axs1.set_title(title_3d)
    axs1.set_xlabel("x")
    axs1.set_ylabel("y")
    axs1.set_zlabel("z")
    axs1.legend()
    axs1.view_init(elev=30, azim=135) # Imposta un angolo di visuale

    # --- GENERAZIONE GRAFICO 2D (a destra) ---
    axs0 = fig.add_subplot(1, 2, 2)

    # Disegna le curve di livello
    c1 = axs0.contour(X_mesh, Y_mesh, Z, levels=50, cmap='viridis', alpha=0.7)
    fig.colorbar(c1, ax=axs0, label='Valore di $f(x,y)$')

    # Disegna il percorso
    axs0.plot(path_x, path_y, 'r-o', markersize=3, label='Percorso Ottimizzazione')

    # Evidenzia il punto iniziale e finale
    axs0.plot(path_x[0], path_y[0], 'go', markersize=10, label=f'Start: ({path_x[0]:.1f}, {path_y[0]:.1f})')
    axs0.plot(path_x[-1], path_y[-1], 'rx', markersize=12, label=f'End: ({path_x[-1]:.2f}, {path_y[-1]:.2f})')
    axs0.plot(tmin[0], tmin[1], 'b*', markersize=15, label=f'Minimo Teorico ({tmin[0]}, {tmin[1]})')

    # Etichette e titoli
    axs0.set_title(title_2d)
    axs0.set_xlabel("x")
    axs0.set_ylabel("y")
    axs0.legend()
    axs0.set_aspect('equal', adjustable='box')

    plt.tight_layout()
    plt.show()


---
## 4. Caso di Studio 1: Funzione Quadratica
$$f(x, y) = (x-5)^2 + (y-2)^2$$

Cerchiamo i punti critici della funzione (punti a gradiente nullo).

$\frac{\partial f}{\partial x} = 2(x-5)$

$\frac{\partial f}{\partial y} = 2(y-2)$

Ponendo entrambe le derivate a zero, troviamo un unico punto critico in $\mathbf{x}^* = (5, 2)$.
Dato che la funzione è una somma di quadrati, questo è l'unico minimo globale ($f(5,2) = 0$).

**Nota:** Per una funzione quadratica, il metodo del Gradiente con line search esatta converge in una sola iterazione. Il backtracking approssima questo comportamento, portando a una convergenza molto rapida.

In [ ]:
# ---- Definizione Funzione 1 ----
def f1(X):
    x = X[0]
    y = X[1]
    return (x - 5)**2 + (y - 2)**2

def df1(X):
    x = X[0]
    y = X[1]
    df_dx = 2 * (x - 5)
    df_dy = 2 * (y - 2)
    return np.array([df_dx, df_dy])

def hess_f1(X):
    # Hessiana costante per una funzione quadratica
    return np.array([ [2.0, 0.0],
                      [0.0, 2.0] ])
    
# ---- Parametri Comuni ----
x_range1 = np.linspace(-2, 12, 100)
y_range1 = np.linspace(-2, 8, 100)
tmin1 = [5.0, 2.0]

maxit = 300
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto_iniziale1 = np.array([0.0, 0.0])

### 4.1 Esecuzione e Confronto Metodi (Caso 1)

In [ ]:
print("--- Metodo del Gradiente (GD) ---")
sol_gd, iter_gd, path_gd, exit_gd = GD(f1, df1, punto_iniziale1, maxit, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_gd}")
print(f"Numero iterazioni: {iter_gd}")
print(f"Condizione di uscita: {exit_gd}")

print("--- Metodo di Newton ---")
sol_newton, iter_newton, path_newton, exit_newton = Newton(f1, df1, hess_f1, punto_iniziale1, maxit, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_newton}")
print(f"Numero iterazioni: {iter_newton}")
print(f"Condizione di uscita: {exit_newton}")

print("--- Metodo del Gradiente Stocastico (Simulato) ---")
sol_sgd, iter_sgd, path_sgd, exit_sgd = SGD(f1, df1, punto_iniziale1, maxit, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_sgd}")
print(f"Numero iterazioni: {iter_sgd}")
print(f"Condizione di uscita: {exit_sgd}")


### 4.2 Grafici (Caso 1)

In [ ]:
titolo = f"Caso 1 (Quadratica) - Metodo del Gradiente ({iter_gd} iterazioni)"
graph_generator(x_range1, y_range1, f1, path_gd, tmin1, 
                title_2d=titolo,
                title_3d=titolo)

In [ ]:
titolo = f"Caso 1 (Quadratica) - Metodo di Newton ({iter_newton} iterazioni)"
graph_generator(x_range1, y_range1, f1, path_newton, tmin1, 
                title_2d=titolo,
                title_3d=titolo)

---
## 5. Caso di Studio 2: Funzione di Rosenbrock
$$f(x, y) = (1-x)^2 + 100(y-x^2)^2$$

Questa è una funzione non-convessa classica, nota per avere una valle parabolica stretta e lunga, che rende difficile la convergenza per i metodi basati solo sul gradiente.

Calcoliamo le derivate parziali:

$\frac{\partial f}{\partial x} = -2(1-x) + 100 \cdot 2(y-x^2)(-2x) = 2(x-1) - 400x(y-x^2)$

$\frac{\partial f}{\partial y} = 100 \cdot 2(y-x^2) = 200(y-x^2)$

Ponendo $\frac{\partial f}{\partial y} = 0$, otteniamo $y=x^2$. Sostituendo nella prima equazione:
$2(x-1) - 400x(x^2-x^2) = 0 \implies 2(x-1) = 0 \implies x=1$.

L'unico punto critico è $(1, 1)$.

L'Hessiana è:
$$H(x,y) = \begin{bmatrix} 
\frac{\partial^2 f}{\partial x^2} & \frac{\partial^2 f}{\partial x \partial y} \\
\frac{\partial^2 f}{\partial y \partial x} & \frac{\partial^2 f}{\partial y^2}
\end{bmatrix} =
\begin{bmatrix} 
 2 - 400y + 1200x^2 & -400x \\
-400x & 200
\end{bmatrix}$$

Valutando nel punto critico $(1, 1)$:
$H(1,1) = \begin{bmatrix} 2 - 400 + 1200 & -400 \\ -400 & 200 \end{bmatrix} = \begin{bmatrix} 802 & -400 \\ -400 & 200 \end{bmatrix}$

Il determinante è $\det(H) = 802 \cdot 200 - (-400)^2 = 160400 - 160000 = 400 > 0$.
Poiché $H_{11} = 802 > 0$ e $\det(H) > 0$, la matrice è definita positiva e il punto $(1, 1)$ è un minimo locale (e globale).

In [ ]:
# ---- Definizione Funzione 2 (Rosenbrock) ----
def f2(X):
    x = X[0]
    y = X[1]
    return (1 - x)**2 + 100 * (y - x**2)**2

def df2(X):
    x = X[0]
    y = X[1]
    df_dx = 2 * (x - 1) - 400 * x * (y - x**2)
    df_dy = 200 * (y - x**2)
    return np.array([df_dx, df_dy])

def hess_f2(X):
    HESS = np.zeros((2, 2))
    x = X[0]
    y = X[1]
    HESS[0, 0] = 2 - 400 * y + 1200 * x**2 # d2f/dx2
    HESS[0, 1] = -400 * x                 # d2f/dxdy
    HESS[1, 0] = -400 * x                 # d2f/dydx
    HESS[1, 1] = 200                      # d2f/dy2
    return HESS
    
# ---- Parametri Comuni ----
x_range2 = np.linspace(-2, 2, 200)
y_range2 = np.linspace(-1, 3, 200)
tmin2 = [1.0, 1.0]

maxit_rosen = 10000 # Aumentiamo le iterazioni massime
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto_iniziale2 = np.array([0.0, 0.0])

### 5.1 Esecuzione e Confronto Metodi (Caso 2)

In [ ]:
print("--- Metodo del Gradiente (GD) ---")
sol_gd_r, iter_gd_r, path_gd_r, exit_gd_r = GD(f2, df2, punto_iniziale2, maxit_rosen, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_gd_r}")
print(f"Numero iterazioni: {iter_gd_r}")
print(f"Condizione di uscita: {exit_gd_r}")

print("--- Metodo di Newton ---")
sol_newton_r, iter_newton_r, path_newton_r, exit_newton_r = Newton(f2, df2, hess_f2, punto_iniziale2, maxit_rosen, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_newton_r}")
print(f"Numero iterazioni: {iter_newton_r}")
print(f"Condizione di uscita: {exit_newton_r}")

print("--- Metodo del Gradiente Stocastico (Simulato) ---")
sol_sgd_r, iter_sgd_r, path_sgd_r, exit_sgd_r = SGD(f2, df2, punto_iniziale2, maxit_rosen, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_sgd_r}")
print(f"Numero iterazioni: {iter_sgd_r}")
print(f"Condizione di uscita: {exit_sgd_r}")


### 5.2 Grafici (Caso 2)

Notare la netta differenza nel numero di iterazioni. Il metodo del Gradiente impiega migliaia di passi, zig-zagando lentamente lungo la valle, mentre il metodo di Newton, utilizzando la curvatura (Hessiana), punta direttamente al minimo in poche iterazioni.

In [ ]:
titolo_r_gd = f"Caso 2 (Rosenbrock) - Metodo del Gradiente ({iter_gd_r} iterazioni)"
graph_generator(x_range2, y_range2, f2, path_gd_r, tmin2, 
                title_2d=titolo_r_gd,
                title_3d=titolo_r_gd)

In [ ]:
titolo_r_n = f"Caso 2 (Rosenbrock) - Metodo di Newton ({iter_newton_r} iterazioni)"
graph_generator(x_range2, y_range2, f2, path_newton_r, tmin2, 
                title_2d=titolo_r_n,
                title_3d=titolo_r_n)

---
## 6. Caso di Studio 3: Funzione Multimodale
$$f(x,y) = x^4 - x^2 + y^2$$

Questa funzione ha più punti critici: due minimi locali (e globali) e un punto di sella.

$\frac{\partial f}{\partial x} = 4x^3 - 2x = 2x(2x^2 - 1)$

$\frac{\partial f}{\partial y} = 2y$

I punti critici si ottengono ponendo il gradiente a zero:
1.  $y=0$
2.  $2x(2x^2 - 1) = 0 \implies x=0$ oppure $x = \pm \sqrt{\frac{1}{2}} \approx \pm 0.707$

Abbiamo tre punti critici: $P_1 = (0, 0)$, $P_2 = (\sqrt{1/2}, 0)$, $P_3 = (-\sqrt{1/2}, 0)$.

L'Hessiana è:
$$H(x,y) = \begin{bmatrix} 
12x^2 - 2 & 0 \\
0 & 2
\end{bmatrix}$$

Analizziamo i punti:

**Punto $P_1(0,0)$:**
$H(0,0) = \begin{bmatrix} -2 & 0 \\ 0 & 2 \end{bmatrix}$. Gli autovalori sono $\lambda_1 = -2, \lambda_2 = 2$. Essendo di segno opposto, $\mathbf{(0,0)}$ **è un punto di sella**.

**Punti $P_2$ e $P_3$ $(\pm\sqrt{1/2}, 0)$:**
$H(\pm\sqrt{1/2}, 0) = \begin{bmatrix} 12(1/2) - 2 & 0 \\ 0 & 2 \end{bmatrix} = \begin{bmatrix} 4 & 0 \\ 0 & 2 \end{bmatrix}$. Gli autovalori sono $\lambda_1 = 4, \lambda_2 = 2$. Essendo entrambi positivi, ${(\pm\sqrt{1/2}, 0)}$ **sono due punti di minimo**.

In [ ]:
# ---- Definizione Funzione 3 (Multimodale) ----
def f3(X):
    x = X[0]
    y = X[1]
    return x**4 - x**2 + y**2

def df3(X):
    x = X[0]
    y = X[1]
    df_dx = 4 * x**3 - 2 * x
    df_dy = 2 * y
    return np.array([df_dx, df_dy])

def hess_f3(X):
    HESS = np.zeros((2, 2))
    x = X[0]
    y = X[1]
    HESS[0, 0] = 12 * x**2 - 2 # d2f/dx2
    HESS[0, 1] = 0               # d2f/dxdy
    HESS[1, 0] = 0               # d2f/dydx
    HESS[1, 1] = 2               # d2f/dy2
    return HESS
    
# ---- Parametri Comuni ----
x_range3 = np.linspace(-1.5, 1.5, 200)
y_range3 = np.linspace(-1.5, 1.5, 200)
# Minimo teorico (il più vicino al punto di partenza)
tmin3 = [np.sqrt(1/2), 0.0]
tmin3_alt = [-np.sqrt(1/2), 0.0]
tsaddle = [0.0, 0.0]

maxit_multi = 500
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto_iniziale3 = np.array([2.0, 1.0]) # Punto di partenza
# Proviamo anche un punto di partenza vicino al punto di sella
punto_iniziale3_sella = np.array([0.1, 0.1])

### 6.1 Esecuzione e Confronto Metodi (Caso 3)

Partendo da $(2.0, 1.0)$, ci aspettiamo che l'algoritmo converga al minimo più vicino, $P_2 = (\sqrt{1/2}, 0)$.

In [ ]:
print("--- Partenza (2.0, 1.0) --- MINIMO P2 ---")
print("--- Metodo del Gradiente (GD) ---")
sol_gd_m, iter_gd_m, path_gd_m, exit_gd_m = GD(f3, df3, punto_iniziale3, maxit_multi, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_gd_m}")
print(f"Numero iterazioni: {iter_gd_m}")
print(f"Condizione di uscita: {exit_gd_m}")

print("--- Metodo di Newton ---")
sol_newton_m, iter_newton_m, path_newton_m, exit_newton_m = Newton(f3, df3, hess_f3, punto_iniziale3, maxit_multi, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_newton_m}")
print(f"Numero iterazioni: {iter_newton_m}")
print(f"Condizione di uscita: {exit_newton_m}")


### 6.2 Grafici (Caso 3 - Partenza 1)

In [ ]:
titolo_m_gd = f"Caso 3 (Multimodale) - Metodo del Gradiente ({iter_gd_m} iterazioni)"
graph_generator(x_range3, y_range3, f3, path_gd_m, tmin3, 
                title_2d=titolo_m_gd,
                title_3d=titolo_m_gd)

In [ ]:
titolo_m_n = f"Caso 3 (Multimodale) - Metodo di Newton ({iter_newton_m} iterazioni)"
graph_generator(x_range3, y_range3, f3, path_newton_m, tmin3, 
                title_2d=titolo_m_n,
                title_3d=titolo_m_n)

### 6.3 Esecuzione (Caso 3 - Partenza vicino alla sella)

Cosa succede se partiamo molto vicini al punto di sella $(0,0)$? L'algoritmo (specialmente Newton) potrebbe avere difficoltà o convergere molto lentamente all'inizio, prima di "cadere" in uno dei due minimi.

In [ ]:
print("--- Partenza (0.1, 0.1) --- VICINO A SELLA ---")
print("--- Metodo del Gradiente (GD) ---")
sol_gd_m2, iter_gd_m2, path_gd_m2, exit_gd_m2 = GD(f3, df3, punto_iniziale3_sella, maxit_multi, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_gd_m2}")
print(f"Numero iterazioni: {iter_gd_m2}")
print(f"Condizione di uscita: {exit_gd_m2}")

print("--- Metodo di Newton ---")
sol_newton_m2, iter_newton_m2, path_newton_m2, exit_newton_m2 = Newton(f3, df3, hess_f3, punto_iniziale3_sella, maxit_multi, tolleranza_f, tolleranza_x)
print(f"Soluzione: {sol_newton_m2}")
print(f"Numero iterazioni: {iter_newton_m2}")
print(f"Condizione di uscita: {exit_newton_m2}")


In [ ]:
titolo_m_n2 = f"Caso 3 (Multimodale) - Newton da (0.1, 0.1) ({iter_newton_m2} iterazioni)"
graph_generator(x_range3, y_range3, f3, path_newton_m2, tmin3, 
                title_2d=titolo_m_n2,
                title_3d=titolo_m_n2)

---
Fine dell'analisi.